In [ ]:
# !pip install -r ./requirements.txt

In [ ]:
# !pip uninstall virny -y

In [ ]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [ ]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

## Import dependencies

In [ ]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSPublicCoverageDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.experiment_interface import run_exp_iter_with_eq_odds

## Define Input Variables

In [ ]:
ROOT_DIR = os.getcwd()
DATASET_NAME = 'ACSPublicCoverageDataset'
FAIRNESS_INTERVENTION_NAME = 'ROC'
EXPERIMENT_NAME = 'ROC_acs_pubcov'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [True]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'folk_CA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

In [ ]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [ ]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'aa7a7273-bf7c-44f9-becc-3b0384b722c5',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

## Initialize custom objects

In [ ]:
data_loader = ACSPublicCoverageDataset(state=['CA'], year=2018, with_nulls=False,
                                       subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

In [ ]:
data_loader.X_data.shape

## Run experiment iterations

### Experiment iteration 1

In [ ]:
# tuned_params_filenames = [
#     'tuning_results_Folktables_CA_2018_Public_Coverage_20240104__230003.csv'
# ]
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
#                                       FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'RandomForestClassifier': models_params_for_tuning['RandomForestClassifier']
}

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          with_tuning=True,
#                           with_tuning=False,
#                           tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 2

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240105__224448.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'RandomForestClassifier': models_params_for_tuning['RandomForestClassifier']
}

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240105__224448.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'RandomForestClassifier': models_params_for_tuning['RandomForestClassifier']
}

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240105__224448.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'RandomForestClassifier': models_params_for_tuning['RandomForestClassifier']
}

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240105__224448.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'RandomForestClassifier': models_params_for_tuning['RandomForestClassifier']
}

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_20240105__224448.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
models_params_for_tuning = {
    'RandomForestClassifier': models_params_for_tuning['RandomForestClassifier']
}

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name=DATASET_NAME,
                          postprocessor_name='ROC',
                          verbose=True)